# Generación de Datos Sintéticos para Sistema NBA

Este notebook genera tres bases de datos sintéticas:
1. Base de clientes
2. Base de productos
3. Base de interacciones cliente-producto

In [ ]:
import numpy as np
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta
import random

# Configurar semilla para reproducibilidad
np.random.seed(42)
random.seed(42)
fake = Faker('es_ES')

## 1. Generación de Base de Clientes

In [ ]:
def generar_base_clientes(n_clientes=1000):
    # Definir segmentos y canales
    segmentos = ['Premium', 'Classic', 'Basic']
    canales = ['Digital', 'Sucursal', 'Call Center', 'App Móvil']
    
    clientes = []
    for _ in range(n_clientes):
        # Generar edad entre 18 y 80 años
        edad = random.randint(18, 80)
        
        # Asignar segmento basado en distribución
        segmento = np.random.choice(segmentos, p=[0.2, 0.5, 0.3])
        
        # Generar ingresos basados en segmento
        if segmento == 'Premium':
            ingresos = random.randint(8000, 20000)
        elif segmento == 'Classic':
            ingresos = random.randint(4000, 8000)
        else:
            ingresos = random.randint(1000, 4000)
        
        # Asignar canal preferido
        canal_preferido = random.choice(canales)
        
        clientes.append({
            'id_cliente': f'CLI_{str(_+1).zfill(6)}',
            'edad': edad,
            'segmento': segmento,
            'ingresos': ingresos,
            'canal_preferido': canal_preferido
        })
    
    return pd.DataFrame(clientes)

# Generar base de clientes
df_clientes = generar_base_clientes()
print("Muestra de la base de clientes:")
display(df_clientes.head())
print(f"\nTotal de clientes generados: {len(df_clientes)}")

## 2. Generación de Base de Productos

In [ ]:
def generar_base_productos(n_productos=50):
    # Definir categorías de productos
    categorias = {
        'Tarjetas de Crédito': (0.15, 0.25),
        'Préstamos Personales': (0.08, 0.15),
        'Inversiones': (0.03, 0.07),
        'Seguros': (0.20, 0.35),
        'Cuentas de Ahorro': (0.02, 0.05)
    }
    
    productos = []
    for _ in range(n_productos):
        categoria = random.choice(list(categorias.keys()))
        margen_min, margen_max = categorias[categoria]
        
        productos.append({
            'id_producto': f'PRD_{str(_+1).zfill(4)}',
            'categoria': categoria,
            'margen_esperado': round(random.uniform(margen_min, margen_max), 4)
        })
    
    return pd.DataFrame(productos)

# Generar base de productos
df_productos = generar_base_productos()
print("Muestra de la base de productos:")
display(df_productos.head())
print(f"\nTotal de productos generados: {len(df_productos)}")

## 3. Generación de Base de Interacciones

In [ ]:
def generar_base_interacciones(df_clientes, df_productos, n_interacciones=5000):
    interacciones = []
    
    for _ in range(n_interacciones):
        # Seleccionar cliente y producto aleatorio
        cliente = df_clientes.sample(1).iloc[0]
        producto = df_productos.sample(1).iloc[0]
        
        # Calcular probabilidad de aceptación basada en segmento y categoría
        prob_base = {
            'Premium': 0.7,
            'Classic': 0.5,
            'Basic': 0.3
        }[cliente['segmento']]
        
        # Ajustar probabilidad según el margen esperado
        prob_ajustada = prob_base * (1 - producto['margen_esperado'])
        
        # Determinar si la oferta fue aceptada
        aceptado = random.random() < prob_ajustada
        
        interacciones.append({
            'id_interaccion': f'INT_{str(_+1).zfill(6)}',
            'id_cliente': cliente['id_cliente'],
            'id_producto': producto['id_producto'],
            'fecha_interaccion': (datetime.now() - timedelta(days=random.randint(0, 365))).strftime('%Y-%m-%d'),
            'canal': cliente['canal_preferido'],
            'aceptado': aceptado
        })
    
    return pd.DataFrame(interacciones)

# Generar base de interacciones
df_interacciones = generar_base_interacciones(df_clientes, df_productos)
print("Muestra de la base de interacciones:")
display(df_interacciones.head())
print(f"\nTotal de interacciones generadas: {len(df_interacciones)}")

## 4. Análisis Básico de los Datos Generados

In [ ]:
# Estadísticas básicas de clientes
print("\nEstadísticas de la base de clientes:")
print(df_clientes.describe())

# Distribución de productos por categoría
print("\nDistribución de productos por categoría:")
print(df_productos['categoria'].value_counts())

# Tasa de aceptación general
tasa_aceptacion = df_interacciones['aceptado'].mean()
print(f"\nTasa de aceptación general: {tasa_aceptacion:.2%}")

# Tasa de aceptación por segmento
df_analisis = pd.merge(df_interacciones, df_clientes[['id_cliente', 'segmento']], on='id_cliente')
print("\nTasa de aceptación por segmento:")
print(df_analisis.groupby('segmento')['aceptado'].mean())

## 5. Guardar los Datos Generados

In [ ]:
df_analisis

In [ ]:
# Guardar los DataFrames en archivos CSV
df_clientes.to_csv('datos_sinteticos/clientes.csv', index=False)
df_productos.to_csv('datos_sinteticos/productos.csv', index=False)
df_interacciones.to_csv('datos_sinteticos/interacciones.csv', index=False)
df_analisis.to_csv('datos_sinteticos/analisis.csv', index=False)
print("Datos guardados exitosamente en la carpeta 'datos_sinteticos'")